In [ ]:
import os
from pathlib import Path
from matplotlib.ticker import PercentFormatter
import openpyxl
from datetime import date

# Download ZIP Files
import requests, zipfile
from io import BytesIO

path = os.getcwd()
os.chdir('/Users/CanonPattillo/OneDrive - Porter/Desktop/Stats and DS/Canon Functions')

from canon_functions import *
import canon_functions as cf

os.chdir(path)
print(path)

pd.set_option('display.max_columns', None)

Import Medicare Cost Report Preprocessed Data

In [ ]:
FYyear = input("Which Fiscal Year (YYYY) would you like to process? >>>  ")

In [ ]:
df = pd.read_csv(f'/Users/CanonPattillo/OneDrive - Porter/Desktop/Intelligence Central/Medicare Cost Reports/Outputs/Processed_{FYyear}_Medicare_Cost_Report.csv'
                , dtype='str')

In [ ]:
df.head()

# Recreate AHD Files

 REQUIRED
 - CMS Certification Number: Provider Number Col
 - Facility Name: S2_1_C1_3
 - System Affiliation: Data gathered by AHD, Home Office Name? S2_1_C1_141
 - State: S2_1_C2_2
 - City: S2_1_C1_2
 - Zip: S2_1_C3_2
 - Address: S2_1_C1_1
 - County: S2_1_C4_2
 - Latitude:
 - Longitude:
 - Type of Control: S2_1_C1_21
 - Type of Facility: Facility Type Col
 - IPPS Discharges to Home:
 - Total Discharges:
 - Total Acute Beds:
 - Number of Staffed Beds (W/S S-3, Part I, line 7 column 2)
 - Total Inpatient Discharges (All Payors)  (W/S S-3, part I, line 14, column 15)
 - Total Patient Days (all payors) (W/S S-3, part I, line 14, column 8). S3_1_C8_14
 - General Med/Surg Beds (S3_1_C2_7)
 

Notes on Other Definitions

- Beds:	W/S S-3, part I, line 7 col 2
- Revenue:	W/S C, part I, line 30 col 6
- Note:  Revenue for swing beds is included with other services.
- Inpatient Days:	W/S S-3, part I, line 7-9 (payors) col 6-32 (components)
- Discharges:	W/S S-3, part I, col 15
- Average Length of Stay:	Calculated:  Days / Discharges
- Average Daily Census:	Calculated:  Days / 365
- Gross Medicare Patient Revenue:	Calculated; sum of the following:
- IP Routine (Part A)	W/S D-3, col 2, lines 30-42
- IP Ancillary (Part A)	W/S D-3, col 2, line 202
- IP (Part B)	[Not reported since 2004]
- OP Ancillary (Part B)	W/S D, part V, col 2-4, line 202
- Gross Medicaid Patient Revenue	W/S S-10, col 1, line 6
- Gross Total Patient Revenue:	Calculated; sum of the following:
- IP Patient Revenue	W/S G-2, part I, col 1, line 28
- OP Patient Revenue	W/S G-2, part I, col 2, line 28
- Gross Other Patient Revenue:	Calculated: Total - Medicare - Medicaid

# QA

In [ ]:
dft = df.copy()

In [ ]:
dft[(dft['Provider Number'] == '454148') & (dft['10 Field Name'] == 'S2_1_C1_2')]

In [ ]:
dft = dft.sort_values('Process Date').drop_duplicates(subset = ['Provider Number','10 Field Name'], 
                                                      keep='first')

Execute Remove Duplicates

In [ ]:
df = df.sort_values('Process Date').drop_duplicates(subset = ['Provider Number','10 Field Name'], 
                                                      keep='first')

## Development

In [ ]:
# # Reduce to Hopkins for Development
# hopkins = ['090005',
#            '103300',
#            '210009',
#            '210022',
#            '210029',
#            '210048'
#             ]

# jh = df[df['Provider Number'].isin(hopkins)].copy().reset_index(drop = True)

Column Variables

In [ ]:
agg_dict = {'Facility Type':'first',
           'Provider Control Type':'first'}
cv = df.groupby(['Provider Number'], as_index=False).agg(agg_dict)

In [ ]:
cv.sample(5)
# cv[cv.duplicated()]

Row Variables

In [ ]:
var_dict = {'S2_1_C1_3':'Facility Name',
'S2_1_C1_141':'System Affiliation',
'S2_1_C2_2':'State',
'S2_1_C1_2':'City',
'S2_1_C3_2':'Zip',
'S2_1_C1_1':'Address',
'S2_1_C4_2':'County',
'S2_1_C1_21':'Type of Control',
'S3_1_C2_7':'Number of Staffed Beds',
'S3_1_C15_14':'Total Inpatient Discharges (All Payers)',
'S3_1_C8_14':'Total Patient Days (All Payers)',
'S3_1_C2_7':'General Med/Surg Beds'}

In [ ]:
rv = df[['Provider Number','10 Field Name', 'Field Description',
       'Value']].copy()

rv = rv[rv['10 Field Name'].isin(list(var_dict.keys()))]

rv = rv.pivot(index='Provider Number', columns=['10 Field Name'
#                                             ,'Field Description'
                                           ], values='Value').reset_index()

rv = rv.rename(columns = var_dict)

rv.columns.name = None

df1 = rv.merge(cv,
               how = 'left',
               left_on = 'Provider Number',
               right_on = 'Provider Number')



In [ ]:
df1.sample()